In [1]:
import pandas as pd
import re
import datetime
import numpy as np
from gpt3 import try_gpt

# 2022 Data Preprocess

In [2]:
RELEVANT_COLUMNS = ['home_team', 'away_team', 'posteam', 'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet', 'play_type', 'yards_gained', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_conv_result', 'away_timeouts_remaining', 'td_team', 'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 'total_home_score', 'total_away_score', 'posteam_score', 'defteam_score', 'score_differential', 'posteam_score_post', 'defteam_score_post', 'score_differential_post', 'punt_blocked', 'first_down_rush', 'first_down_pass', 'first_down_penalty', 'third_down_converted', 'third_down_failed', 'fourth_down_converted', 'fourth_down_failed', 'incomplete_pass', 'touchback', 'interception', 'punt_inside_twenty', 'punt_in_endzone', 'punt_out_of_bounds', 'punt_downed', 'punt_fair_catch', 'kickoff_inside_twenty', 'kickoff_in_endzone', 'kickoff_out_of_bounds', 'kickoff_downed', 'kickoff_fair_catch', 'fumble_forced', 'fumble_not_forced', 'fumble_out_of_bounds', 'solo_tackle', 'safety', 'penalty', 'tackled_for_loss', 'fumble_lost', 'own_kickoff_recovery', 'own_kickoff_recovery_td', 'qb_hit', 'rush_attempt', 'pass_attempt', 'sack', 'touchdown', 'pass_touchdown', 'rush_touchdown', 'return_touchdown', 'extra_point_attempt', 'two_point_attempt', 'field_goal_attempt', 'kickoff_attempt', 'punt_attempt', 'fumble', 'complete_pass', 'assist_tackle', 'lateral_reception', 'lateral_rush', 'lateral_return', 'lateral_recovery', 'passer_player_id', 'passer_player_name', 'receiver_player_id', 'receiver_player_name', 'rusher_player_id', 'rusher_player_name', 'lateral_receiver_player_id', 'lateral_receiver_player_name', 'lateral_rusher_player_id', 'lateral_rusher_player_name', 'lateral_sack_player_id', 'lateral_sack_player_name', 'interception_player_id', 'interception_player_name', 'lateral_interception_player_id', 'lateral_interception_player_name', 'punt_returner_player_id', 'punt_returner_player_name', 'lateral_punt_returner_player_id', 'lateral_punt_returner_player_name', 'kickoff_returner_player_name', 'kickoff_returner_player_id', 'lateral_kickoff_returner_player_id', 'lateral_kickoff_returner_player_name', 'punter_player_id', 'punter_player_name', 'kicker_player_name', 'kicker_player_id', 'own_kickoff_recovery_player_id', 'own_kickoff_recovery_player_name', 'blocked_player_id', 'blocked_player_name', 'tackle_for_loss_1_player_id', 'tackle_for_loss_1_player_name', 'tackle_for_loss_2_player_id', 'tackle_for_loss_2_player_name', 'qb_hit_1_player_id', 'qb_hit_1_player_name', 'qb_hit_2_player_id', 'qb_hit_2_player_name', 'forced_fumble_player_1_team', 'forced_fumble_player_1_player_id', 'forced_fumble_player_1_player_name', 'forced_fumble_player_2_team', 'forced_fumble_player_2_player_id', 'forced_fumble_player_2_player_name', 'solo_tackle_1_team', 'solo_tackle_2_team', 'solo_tackle_1_player_id', 'solo_tackle_2_player_id', 'solo_tackle_1_player_name', 'solo_tackle_2_player_name', 'assist_tackle_1_player_id', 'assist_tackle_1_player_name', 'assist_tackle_1_team', 'assist_tackle_2_player_id', 'assist_tackle_2_player_name', 'assist_tackle_2_team', 'assist_tackle_3_player_id', 'assist_tackle_3_player_name', 'assist_tackle_3_team', 'assist_tackle_4_player_id', 'assist_tackle_4_player_name', 'assist_tackle_4_team', 'pass_defense_1_player_id', 'pass_defense_1_player_name', 'pass_defense_2_player_id', 'pass_defense_2_player_name', 'fumbled_1_team', 'fumbled_1_player_id', 'fumbled_1_player_name', 'fumbled_2_player_id', 'fumbled_2_player_name', 'fumbled_2_team', 'fumble_recovery_1_team', 'fumble_recovery_1_yards', 'fumble_recovery_1_player_id', 'fumble_recovery_1_player_name', 'fumble_recovery_2_team', 'fumble_recovery_2_yards', 'fumble_recovery_2_player_id', 'fumble_recovery_2_player_name', 'return_team', 'return_yards', 'penalty_team', 'penalty_player_id', 'penalty_player_name', 'penalty_yards', 'replay_or_challenge', 'replay_or_challenge_result', 'penalty_type', 'defensive_two_point_attempt', 'defensive_two_point_conv', 'defensive_extra_point_attempt', 'defensive_extra_point_conv']

In [3]:
pbp_data_22 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2022.csv")

C:\Users\costi\AppData\Local\Temp\ipykernel_22128\813836328.py:1: DtypeWarning: Columns (179,180,182,183,189,190,193,194,197,198,203,204,205,206,218,219,220,243,244,245,253,254,255,260,262,263,283,284,301) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_data_22 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2022.csv")


In [4]:
highlights22 = pd.read_csv("./transcripts_2022.csv")
highlights22 = highlights22[~highlights22["interval"].isna()]

In [5]:
def process_inning_time(time_str):
    quarter = time_str[0]
    if quarter == "O":
        time_remaining = 0
    else:
        time_remaining = (4 - int(quarter)) * 900
    times = re.findall("\d?\d:\d\d",time_str)
    timedeltas = pd.to_timedelta(pd.to_datetime(times,format='%M:%S').strftime('%H:%M:%S').values)
    timelists = (timedeltas + datetime.timedelta(seconds=time_remaining)).seconds
    return pd.Interval(timelists[1],timelists[0],closed="both")

In [6]:
highlights22["intervals"] = highlights22["interval"].apply(process_inning_time)
highlights22["date"] = pd.to_datetime(highlights22["date"])
pbp_data_22["game_date"] = pd.to_datetime(pbp_data_22["game_date"])

In [7]:
merged22 = pbp_data_22.merge(highlights22,left_on=["home_team","away_team","game_date"], right_on=["home","away","date"])

In [8]:
def in_interval(row):
    return row["game_seconds_remaining"] in row["intervals"]

In [9]:
plays22 = merged22[merged22.apply(lambda x: x["game_seconds_remaining"] in x["intervals"],axis=1)]

In [10]:
touches = plays22[plays22["touchdown"] == 1].groupby("intervals")['touchdown'].sum()

In [11]:
touchdown_index = touches[touches == 1].index

In [12]:
plays22 = plays22[(plays22["touchdown"] == 1) & (plays22["intervals"].isin(touchdown_index))]

In [13]:
X = plays22[RELEVANT_COLUMNS+["transcript"]]

In [14]:
X.shape

(31, 198)

In [15]:
ret_list = []
for i in range(X.shape[0]):
    ret_list += [{"json":X[RELEVANT_COLUMNS].iloc[i].replace(0,np.nan).dropna().to_json(),
                  "transcript":X["transcript"].iloc[i]}]

In [21]:
gpt22 = pd.DataFrame(ret_list)

In [24]:
gpt22["commentary"] = gpt22["json"].apply(try_gpt)

RateLimitError: Rate limit reached for default-text-davinci-002 in organization org-ZsQyOjrRy2ooz56rLLjBHdxy on requests per min. Limit: 60.000000 / min. Current: 70.000000 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://beta.openai.com/account/billing to add a payment method.

In [36]:
group1 = gpt22[:10]
group2 = gpt22[10:20]
group3 = gpt22[20:]

In [40]:
group1["commentary"] = group1["json"].apply(try_gpt)
group1

C:\Users\costi\AppData\Local\Temp\ipykernel_22128\2671908567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group1["commentary"] = group1["json"].apply(try_gpt)


,json,transcript,commentary
0,"{""home_team"":""HOU"",""away_team"":""CLE"",""posteam""...",What a punt here by Johnson. Peoples-jones. Go...,\n\nPunting away here on 4th and 2. Houston wi...
1,"{""home_team"":""HOU"",""away_team"":""CLE"",""posteam""...",On 2nd and 10. Allen trying to catch some brea...,\n\nSecond and 10. Deshaun Watson drops back. ...
2,"{""home_team"":""HOU"",""away_team"":""CLE"",""posteam""...",Browns fans hoping to get a little help. That ...,\n\nFirst and 10. Allen in the shotgun. He'll ...
3,"{""home_team"":""CHI"",""away_team"":""GB"",""posteam"":...","Fields, oh what a move to move past Nixon. And...",\n\nFields with the handoff. He'll take it aro...
4,"{""home_team"":""CHI"",""away_team"":""GB"",""posteam"":...",Hit to Montgomery. With a nice cut. And he pow...,\n\nMontgomery takes the handoff and he's look...
5,"{""home_team"":""CHI"",""away_team"":""GB"",""posteam"":...",He'll snap it on fourth down. Rodgers steps up...,\n\nFourth and four. Rodgers drops back. He's ...
6,"{""home_team"":""DAL"",""away_team"":""IND"",""posteam""...","Here is Ryan, firing into the endzone. Open fo...",\n\nA.Dulin takes it in for the touchdown! Rya...
7,"{""home_team"":""DAL"",""away_team"":""IND"",""posteam""...",And Dak will give it to Pollard who walks it in!,\nPollard! He'll take it in for the touchdown!...
8,"{""home_team"":""DAL"",""away_team"":""IND"",""posteam""...","And Prescott gets away from Kwity Paye, throws...",\n\nGallup with the catch! He's got the first ...
9,"{""home_team"":""CIN"",""away_team"":""KC"",""posteam"":...","Second and goal from the four, quarterback dra...",\nBurrow. He'll hand it off to Mixon on the di...


In [41]:
group2["commentary"] = group2["json"].apply(try_gpt)
group2

C:\Users\costi\AppData\Local\Temp\ipykernel_22128\1455296522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group2["commentary"] = group2["json"].apply(try_gpt)


,json,transcript,commentary
10,"{""home_team"":""CIN"",""away_team"":""KC"",""posteam"":...","Could be dangerous right here, third and eight...",\nBurrow with a great pass to Higgins who make...
11,"{""home_team"":""CIN"",""away_team"":""KC"",""posteam"":...","Mahomes completes it, McKinnon for the touchdown!",\n\nMahomes will flip it to McKinnon on the ru...
12,"{""home_team"":""LV"",""away_team"":""LAC"",""posteam"":...",But how to getting him up on the field opens t...,\n\nCarr. looking for Hollins in the end zone....
13,"{""home_team"":""LV"",""away_team"":""LAC"",""posteam"":...","Jacobs in the backfield. Here are the Raiders,...",\n\nCarr looking deep for Adams and he's got h...
14,"{""home_team"":""LV"",""away_team"":""LAC"",""posteam"":...",You can just tell that he is in the zone and h...,\n\nFirst and 10 for Las Vegas. Carr. He'll dr...
15,"{""home_team"":""LV"",""away_team"":""LAC"",""posteam"":...",The good thing is that you have seen him in hi...,\nHerbert will take the snap. Looks to his rig...
16,"{""home_team"":""SF"",""away_team"":""MIA"",""posteam"":...",This explosive offense gets things going with ...,\n\nAnd the first play from scrimmage Tagovail...
17,"{""home_team"":""SF"",""away_team"":""MIA"",""posteam"":...","Down to two, they get it off, Purdy looks to t...",\n\nPurdy on first and goal. He'll look to his...
18,"{""home_team"":""SF"",""away_team"":""MIA"",""posteam"":...",So here we go on third down and goal in a ten-...,\n\nPurdy completes that ball to Christian McC...
19,"{""home_team"":""MIN"",""away_team"":""NYJ"",""posteam""...","First and goal. Handoff, Cook, the stutterstep...",\nFirst and goal. From the four yard line it's...


In [47]:
group3["commentary"] = group3["json"].apply(try_gpt)
group3

C:\Users\costi\AppData\Local\Temp\ipykernel_22128\4072402062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group3["commentary"] = group3["json"].apply(try_gpt)


,json,transcript,commentary
20,"{""home_team"":""MIN"",""away_team"":""NYJ"",""posteam""...",First down for Minnesota at the 14 yard line. ...,\n\nAnd Mattison will take it all the way! He'...
21,"{""home_team"":""MIN"",""away_team"":""NYJ"",""posteam""...",It's third down. Is it Jefferson time? Cousins...,\n\nCousins back to pass. He's looking for Jef...
22,"{""home_team"":""LA"",""away_team"":""SEA"",""posteam"":...","Powell in motion, the give to Akers, trying to...",\n\nAkers with the handoff. He'll take it into...
23,"{""home_team"":""LA"",""away_team"":""SEA"",""posteam"":...","Rams bring pressure, Smith gets a pocket clean...",\nThird and 7. Smith in the gun. He'll look to...
24,"{""home_team"":""LA"",""away_team"":""SEA"",""posteam"":...","Third and goal at the four, Geno clapping for ...",\n\nFirst and goal. Smith with a short pass to...
25,"{""home_team"":""PHI"",""away_team"":""TEN"",""posteam""...",Hurts again all day. Going deep for the end zo...,\nSecond and 10. Hurts in the shotgun. He'll l...
26,"{""home_team"":""PHI"",""away_team"":""TEN"",""posteam""...",This is where Tennessee wants to live. These d...,\n\nRoughness on the play. 15 yard penalty aga...
27,"{""home_team"":""PHI"",""away_team"":""TEN"",""posteam""...","Hurts, lofting deep again, Brown is wide open ...",\nSecond and 15. Hurts going deep to Alshon Je...
28,"{""home_team"":""NYG"",""away_team"":""WAS"",""posteam""...",Make sure that he's gonna confuse that offensi...,\n\nFirst and 10 for the Redskins. Heinicke wi...
29,"{""home_team"":""NYG"",""away_team"":""WAS"",""posteam""...",Jones with 48 yards on the ground today. Hands...,\n\nSaquon Barkley with the handoff. He'll mak...


In [49]:
final_sheet = pd.concat([group1,group2,group3])

In [138]:
final_sheet["gpt"] = final_sheet["commentary"].str.replace("\n+"," ").str.strip()
final_sheet["real_transcript"] = final_sheet["transcript"].str.replace("\n+"," ").str.strip()

C:\Users\costi\AppData\Local\Temp\ipykernel_22128\784336393.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  final_sheet["gpt"] = final_sheet["commentary"].str.replace("\n+"," ").str.strip()
C:\Users\costi\AppData\Local\Temp\ipykernel_22128\784336393.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  final_sheet["real_transcript"] = final_sheet["transcript"].str.replace("\n+"," ").str.strip()


In [143]:
final_sheet[["real_transcript","gpt"]].to_csv("blind_test_solutions.csv",index=False)

In [140]:
shuffled = final_sheet[["real_transcript","gpt"]].apply(lambda x : x.sample(frac=1).reset_index(drop=True),axis=1)

In [144]:
shuffled.to_csv("blind_test.csv",index=False)

In [142]:
shuffled.iloc[0][1]

"Punting away here on 4th and 2. Houston will give it back to the Browns inside their own 10 yard line. People-Jones takes it on the run! He's got some open field in front of him! Peoples-Jones will take it all the way for the touchdown!!"